## Analyse Autapse run

In [ ]:
%matplotlib inline
from __future__ import division
import numpy as np
import scipy.signal as ss
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pypolyagamma import PyPolyaGamma
from math import factorial

import likelihoodfree.io as io
import likelihoodfree.viz as viz
import likelihoodfree.PDF as lfpdf
import os

## Some parameters of the analysis

In [ ]:
# to save or not to save plots
save_plot_flag = 0

###############################################################################
# number of parameters inferred
num_param_inf = 1

###############################################################################
# number of simulations per iteration
sim_step = 100
num_round = 5
num_sim = np.linspace(sim_step, num_round*sim_step, num_round).astype(int)

###############################################################################
# colors
col_original = (35/255,86/255,167/255)
col_lfree = (0, 174/255,239/255)

## Loading files

In [ ]:
# loading filenames, paths and data
iw_loss_ind = 0
prefix = 'autapse_test_iw'
basepath = '../results/autapse/'
dir_nets = basepath + 'nets/'
dir_plots = basepath + 'plots/'

# create folder for plots
if save_plot_flag:
    if not os.path.exists(dir_plots):
        os.makedirs(dir_plots)

###############################################################################
# loading data and posteriors
dists, infos, losses, nets, posteriors, sims = io.load_prefix(dir_nets, prefix)

# data
l_sims = list(sims.items())
obs_stats = l_sims[len(sims)-1][1].obs
y_obs = l_sims[len(sims)-1][1].obs_trace.reshape(-1,1)

# posterior
l_posteriors = list(posteriors.items())
posterior = l_posteriors[len(l_posteriors)-1][1]

###############################################################################
# ground-truth parameters
gt = l_sims[0][1].true_params

###############################################################################
# time
t = l_sims[0][1].t

###############################################################################
# input
I = l_sims[0][1].I_obs
I = I.reshape(1,-1)

## Posterior distribution

In [ ]:
levels=(0.68, 0.95)

alpha_rep = np.linspace(0.5,1,num_rep)

for i in range(len(l_posteriors)):
    
    f, axs = plt.subplots(1,3,figsize=(np.maximum(5,1.5*posterior.ndim),
                                   np.maximum(5,1.5*posterior.ndim)))
    posterior = l_posteriors[i][1]
    for ii in range(posterior.ndim):
        for j in range(ii,posterior.ndim):

            if ii == j:
                ax = plt.subplot2grid((posterior.ndim, posterior.ndim), (ii,j))

                lims = np.array([0.5*gt[ii],1.5*gt[ii]])
                xx = np.linspace(lims[0], lims[1], 500)

                pp = posterior.eval(xx, ii=[ii], log=False)
                ax.plot(xx, pp, lw=1, color=col_lfree)
                ax.vlines(gt[ii], 0, ax.get_ylim()[1], lw=1, color='r')
                
                if ii==0 and j==0:
                    plt.title('round '+str(i+1))

            else:
                ax = plt.subplot2grid((posterior.ndim, posterior.ndim), (ii,j))
                
                lims = np.array([[0.5*gt[ii],1.5*gt[ii]],[0.5*gt[j],
                                  0.5*gt[j]]])

                xx = np.linspace(lims[0, 0], lims[0, 1], 500)
                yy = np.linspace(lims[1 ,0], lims[1, 1], 500)
                X, Y = np.meshgrid(xx, yy)
                xy = np.concatenate([X.reshape([-1, 1]), Y.reshape([-1, 1])], axis=1)

                pp = posterior.eval(xy, ii=[ii, j], log=False)
                pp = pp.reshape(list(X.shape))
                ax.contour(X, Y, viz.probs2contours(pp, levels), levels, lw=1,colors=('w','y'))
                ax.set_xlim(lims[0])
                ax.set_ylim(lims[1])
                ax.plot(gt[ii], gt[j], 'r.', ms=8)
    
    plt.tight_layout()

# save figure
if save_plot_flag == 1:
    plt.savefig(dir_plots + 'fig_summary_lfree_stability' + prefix + '.pdf')
plt.show()

## Training loss

In [ ]:
i=0
for k, v in losses.items():
    i = i+1
    fig, ax = viz.loss(v, title=k)
#     plt.show()
    plt.title('round '+str(i))

## Distance of samples

In [ ]:
for k, v in dists.items():
    fig, ax = viz.dist(v, title=k)

## Validation set

In [ ]:
for i in range(num_round):
    plt.plot(io.nth(losses, i)['val_val'],label='round '+str(i+1))
    plt.ylim([0,20])
plt.legend()

## Importance weights

In [ ]:
f, axs = plt.subplots(1,2,figsize=(15,5))

for i in range(num_round):    
    ax = plt.subplot(1,2,1)
    plt.semilogy(io.nth(losses, i)['trn_out'][-1][1],'.')
    
    ax = plt.subplot(1,2,2)
    plt.hist(io.nth(losses, i)['trn_out'][-1][1],bins=20,log=True,alpha=0.5,label='round '+str(i+1))

plt.legend()

In [ ]:
len(io.nth(losses, i)['trn_out'][-1][1])

## Info

In [ ]:
for k, v in infos.items():
    print(viz.info(v, title=k))